In [1]:
import os
import json
import math
import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize
import matplotlib
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union
import numpy as np
from PIL import Image
from numpy import newaxis

import matplotlib.pyplot as plt


In [ ]:
#ff = np.ones([3,3,3])
#np.remainder(256*ff,256*np.ones([3,3,3]))

### Open image with rasterio:

In [ ]:
#with rasterio.open('/home/odemoly/Documents/Supaero 4A/PIE/Airport_Feature_Auto_Detection/GeoTIFF/LFMP_ortho.tif') as src:
#    img = src.read()
#    meta = src.meta

#img = reshape_as_image(img)


#plt.imshow(img)

### Read Geojson as  a dataframe:

In [ ]:
def polygon_df(filepath):
    df = gpd.read_file(filepath)


    df = df.drop(columns=['originated', 'readonly',
           'notvalidated', 'lock', 'elev', 'hacc', 'iata', 'name', 'idarpt',
           'acft', 'idnumber', 'termref', 'pcn', 'restacft', 'status', 'surftype',
           'length', 'width', 'color', 'direc', 'style', 'rwymktyp', 'asda',
           'availPavedSurfFromThr', 'brngmag', 'brngtrue', 'cat', 'ellipse',
           'geound', 'lda', 'rops_landing_length', 'rwyslope', 'tdze', 'tdzslope',
           'thrtype', 'toda', 'tora', 'vasis', 'bridge', 'gsurftyp', 'runwayexit',
           'imagery_date'])

    
    return df[df["geometry"].geom_type == 'Polygon']

In [2]:
def polygon_df(filepath):
    df = gpd.read_file(filepath)

    #df = df.drop(columns=['originated', 'readonly',
    #       'notvalidated', 'lock', 'elev', 'hacc', 'iata', 'name', 'idarpt',
    #       'acft', 'idnumber', 'termref', 'pcn', 'restacft', 'status', 'surftype',
    #       'length', 'width', 'color', 'direc', 'style', 'rwymktyp', 'asda',
    #       'availPavedSurfFromThr', 'brngmag', 'brngtrue', 'cat', 'ellipse',
    #       'geound', 'lda', 'rops_landing_length', 'rwyslope', 'tdze', 'tdzslope',
    #       'thrtype', 'toda', 'tora', 'vasis', 'bridge', 'gsurftyp', 'runwayexit',
    #       'imagery_date'])
    
    df = df.drop(columns=['originated', 'readonly',
           'notvalidated', 'lock', 'elev', 'hacc', 'iata', 'name', 'idarpt',
           'pcn', 'restacft', 'status', 'surftype',
           'length', 'width', 'color', 'direc', 'style', 'rwymktyp', 'asda',
           'availPavedSurfFromThr', 'brngmag', 'brngtrue', 'cat', 'ellipse',
           'geound', 'lda', 'rops_landing_length', 'rwyslope', 'tdze', 'tdzslope',
           'thrtype', 'toda', 'tora', 'vasis', 'bridge', 'gsurftyp', 'runwayexit',
           'imagery_date'])
    
    return df[(df["geometry"].geom_type == 'Polygon')]# | (df["geometry"].geom_type == 'LineString')]

#### Only polygon:

In [ ]:
#df = polygon_df('/home/odemoly/Documents/Supaero 4A/PIE/Airport_Feature_Auto_Detection/GeoJson/product_ADBLucem_VHHH.json')

In [ ]:
#features_list = []
#for e in df['feattype']:
#    if e['value'] not in features_list:
#        features_list.append(e['value'])

#print(features_list)

In [ ]:
#k = 0
#while k<1: 
#    k+=1
#    #df[df['feattype']]# == {'value': 'kjfg'}].geometry
#    for g in df[df['feattype'] == {'value': 'Runway.runwayexitline'}].geometry.values:
#        print(g)
#        print(poly_from_utm(g, src.meta['transform']))
    

In [3]:
def poly_from_utm(polygon, transform):
    poly_pts = []
    
    # make a polygon from multipolygon
    poly = cascaded_union(polygon)
    for i in np.array(poly.exterior.coords):
        
        # transfrom polygon to image crs, using raster meta
        poly_pts.append(~transform * tuple(i))
        
    # make a shapely Polygon object
    new_poly = Polygon(poly_pts)
    return new_poly

# creating binary mask for field/not_filed segmentation.

def create_mask(features_list,src,df):
    im_size = (src.meta['height'], src.meta['width'])
    whole_mask = np.zeros([3,src.meta['height'],src.meta['width']])
    #print(np.shape(whole_mask))
    for i,f in enumerate(features_list):
        #print(f)
        poly_shp = []
        data_geom = df[df['feattype'] == {'value': f}].geometry
        if (not df[df['feattype'] == {'value': f}].empty):
            for g in data_geom.values:


                poly = poly_from_utm(g, src.meta['transform'])
                poly_shp.append(poly)

                if (g.geom_type == 'Polygon'):# | g.geom_type == 'LineString'):
                    poly = poly_from_utm(g, src.meta['transform'])
                    poly_shp.append(poly)
                else:
                    for p in g:
                        poly = poly_from_utm(p, src.meta['transform'])
                        poly_shp.append(poly)

            #print('mask')
            #print(poly_shp)
            #print(im_size)
            #print(rasterize(shapes=poly_shp,out_shape=im_size))
            #np.savetxt('rasterize_'+f+'.csv', rasterize(shapes=poly_shp,out_shape=im_size),delimiter=',')
            #print('premask')
            mask = rasterize(shapes=poly_shp,out_shape=im_size)#*color_mapping[f]
            #print('mask_shape = ',np.shape(mask))
            mask_3d = mask[newaxis,:,:]
            mask_3d = mask_3d.astype('uint8')
            #print('mask_3d shape = ',np.shape(mask_3d))
            new_mask = np.concatenate((mask_3d*color_mapping[f][0],mask_3d*color_mapping[f][1],mask_3d*color_mapping[f][2]),axis = 0)
            #print('new_mask shape =', np.shape(new_mask))
            #print('whole_mask shape =', np.shape(whole_mask))
            whole_mask+=new_mask
            #whole_mask = whole_mask + new_mask #%255
            #print('sumok') 
    return whole_mask




def poly_from_utm(polygon, transform):
    poly_pts = []
    
    # make a polygon from multipolygon
    poly = cascaded_union(polygon)
    for i in np.array(poly.exterior.coords):
        
        # transfrom polygon to image crs, using raster meta
        poly_pts.append(~transform * tuple(i))
        
    # make a shapely Polygon object
    new_poly = Polygon(poly_pts)
    return new_poly

# creating binary mask for field/not_filed segmentation.

def create_mask(features_list,src,df):
    im_size = (src.meta['height'], src.meta['width'])
    whole_mask = np.zeros([src.meta['height'],src.meta['width']])

    for i,f in enumerate(features_list[:-1]):
        print(f)
        poly_shp = []
        data_geom = df[df['feattype'] == {'value': f}].geometry
        print(data_geom.values)
        if (not df[df['feattype'] == {'value': f}].empty):
            for g in data_geom.values:
                #print("2d for")


                poly = poly_from_utm(g, src.meta['transform'])
                poly_shp.append(poly)

                if (g.geom_type == 'Polygon' | g.geom_type == 'LineString'):
                    poly = poly_from_utm(g, src.meta['transform'])
                    poly_shp.append(poly)
                else:
                    for p in g:
                        poly = poly_from_utm(p, src.meta['transform'])
                        poly_shp.append(poly)


            mask = rasterize(shapes=poly_shp,out_shape=im_size)*color_mapping[f]
            print("maskok")
            whole_mask = whole_mask + mask
            print("wholemaskok")
    
    return whole_mask

In [4]:
#TO ADAPT ACCORDING TO YOUR PATH TO FOLDER
outfolder = "/home/odemoly/Documents/Supaero 4A/PIE/Airport_Feature_Auto_Detection/Masks/"  #folder where the mask is stored
os.makedirs(outfolder, exist_ok=True)

color_mapping = {'Runway.runwayelement': 80,
 'Runway.runwaymarking': 80,
 'Apron.apronelement': 60,
 'Runway.runwayshoulder': 80,
 'Runway.runwayexitline':80,
 'ServiceRoad.serviceroad': 100,
 'Runway.runwaydisplacedarea': 120}

In [5]:
color_mapping = {'Runway.runwayelement': [255, 0, 0],
 'Runway.runwaymarking': [255, 0, 0],
 'Runway.runwayexitline': [255,0,0],
 'Apron.apronelement': [0, 255, 0],
 'Runway.runwayshoulder': [255, 0, 0],
 'ServiceRoad.serviceroad': [0, 0, 255],
 'Runway.runwaydisplacedarea': [255, 0, 0]}

In [ ]:
#type(color_mapping['Runway.runwaymarking'][0])

In [6]:
features_list = ['Runway.runwayelement', 'Runway.runwaymarking', 'Apron.apronelement','Runway.runwayshoulder','Runway.runwaydisplacedarea']#,'Runway.runwayexitline']#,'ServiceRoad.serviceroad']

In [7]:
import os

directory = os.fsencode("/home/odemoly/Documents/Supaero 4A/PIE/Airport_Feature_Auto_Detection/GeoTIFF")

for file in os.listdir(directory):
    
    filename = os.fsdecode(file)
    if filename.endswith("LFMP_ortho.tif"):
    
    #if filename.endswith(".tif"):
        #print(filename)
        with rasterio.open("/home/odemoly/Documents/Supaero 4A/PIE/Airport_Feature_Auto_Detection/GeoTIFF/{}".format(filename)) as src:
            img = src.read()
            meta = src.meta
        
            filepath = '/home/odemoly/Documents/Supaero 4A/PIE/Airport_Feature_Auto_Detection/GeoJson/product_ADBLucem_{}.json'.format(filename.split('_')[0])
        try:
            #one_tab = np.ones([3,src.meta['height'],src.meta['width']])
            print(filepath)
            df = polygon_df(filepath)
            #print('dfok')
            whole_mask = create_mask(features_list,src,df)
            #print(np.shape(one_tab))
            #print('wholemaskok')
            #a = np.remainder(whole_mask,256*one_tab)
            a = whole_mask
            #print('aok')
            
            #arr = img*a
            img3 = cv2.merge((a[2], a[1], a[0]))  # Use opencv to merge as b,g,r
            cv2.imwrite(outfolder+'mask_colors_{}.jpg'.format(filename.split('_')[0]), img3) 
            #matplotlib.image.imsave(outfolder + 'mask_{}.jpg'.format(filename.split('_')[0]), whole_mask)
            print("image saved")
            
        except:
            pass
    


/home/odemoly/Documents/Supaero 4A/PIE/Airport_Feature_Auto_Detection/GeoJson/product_ADBLucem_LFMP.json
image saved


In [ ]:
#np.shape(whole_mask)



import os

directory = os.fsencode("/home/odemoly/Documents/Supaero 4A/PIE/Airport_Feature_Auto_Detection/GeoTIFF")

for file in os.listdir(directory):
    
    filename = os.fsdecode(file)
    if filename.endswith("CYEG_ortho.tif"):
        print(filename)
        with rasterio.open("/home/odemoly/Documents/Supaero 4A/PIE/Airport_Feature_Auto_Detection/GeoTIFF/{}".format(filename)) as src:
            img = src.read()
            meta = src.meta
        
            filepath = '/home/odemoly/Documents/Supaero 4A/PIE/Airport_Feature_Auto_Detection/GeoJson/product_ADBLucem_{}.json'.format(filename.split('_')[0])
        try:
        
            df = polygon_df(filepath)
            print('dfok')
            whole_mask = create_mask(features_list,src,df)
            print("wholemaskok")
            a = whole_mask
            
            matplotlib.image.imsave(outfolder + 'mask_colors_{}.jpg'.format(filename.split('_')[0]), whole_mask)
            print("image saved")
            
        except:
            pass



In [ ]:
#np.shape(img)

In [ ]:
#print(whole_mask[1,3500,3000])

In [ ]:
#img = rasterio.open('/home/odemoly/Documents/Supaero 4A/PIE/pie-repo/mask_CYEG.jpg')

                   

In [ ]:
#whole_mask_3d = np.array([whole_mask,whole_mask,whole_mask])

from numpy import newaxis
whole_mask_3d = whole_mask[newaxis,:,:]
np.shape(whole_mask_3d)
whole_mask_3d = whole_mask_3d.astype('uint8')
b = np.concatenate((whole_mask_3d,whole_mask_3d,whole_mask_3d),axis = 0)
print(np.shape(b))

print(type(whole_mask[500,400]))
print(type(img[0,500,400]))
print(type(b[0,500,400]))
print(type(whole_mask))
print(type(img))
print(np.shape(b))

print(filename)
np.shape(b)
img = np.array(img)
b=np.array(b)
np.shape(img*b)


test = np.zeros([3,4,3])
np.shape(test)

img2 = Image.fromarray(img,'RGB')
img2.save(outfolder + 'img_test_CYEG.jpg')

import cv2
import numpy as np
arr = img*b
img3 = cv2.merge((arr[2], arr[1], arr[0]))  # Use opencv to merge as b,g,r
cv2.imwrite(outfolder+'out.png', img3) 

matplotlib.image.imsave(outfolder + 'whole_mask_test_{}.jpg'.format(filename.split('_')[0]), img*b)
            

tab = np.ones(3,np.shape(whole_mask))
np.shape(tab)